In [2]:
import pandas as pd
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
from torch.autograd import Variable
import os 

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(10,6,(1,4),stride=(1,1))#(10 - 1)/1 + 1 = 10   (120 - 4)/1 + 1 = 118 118/2 = 59
        self.conv2 = nn.Conv2d(6,10,(1,3),stride=(1,2))#(10 - 2)/2 + 1 = 5   (59 - 3)/1 + 1 = 57 
        self.fc1 = nn.Linear(10*1*58,4)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.xavier_uniform_(m.weight)
                init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                init.normal_(m.weight, std=0.01)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x

def Normlize(Z):
    Zmax, Zmin = Z.max(axis=1), Z.min(axis=1)
    Zmean = Z.mean(axis=1)
    #按列排序
    Zmax, Zmin = Zmax.reshape(-1, 1), Zmin.reshape(-1, 1)
    Zmean = Zmean.reshape(-1, 1)
    Z = (Z - Zmean) / (Zmax - Zmin)
    return Z


def Data_Reading(Normalization=True):
    train_x = np.load('F:\\gitworkspace\\python\\e-nose-cnn\\train\\trainset2.npy')
    train_y = np.load('F:\\gitworkspace\\python\\e-nose-cnn\\train\\trainlabel2.npy')
    test_x = np.load('F:\\gitworkspace\\python\\e-nose-cnn\\test\\testset2.npy')
    test_y = np.load('F:\\gitworkspace\\python\\e-nose-cnn\\test\\testlabel2.npy')

    # Normalization
    train_x_Normed = Normlize(train_x)
    test_x_Normed = Normlize(test_x)

    # xlsx to tensor
    if Normalization:
        train_x = torch.from_numpy(train_x_Normed).type(torch.cuda.FloatTensor)
        test_x = torch.from_numpy(test_x_Normed).type(torch.cuda.FloatTensor)
        train_y = torch.from_numpy(train_y).type(torch.int64)
        test_y = torch.from_numpy(test_y).type(torch.int64)

    else:
        train_x = torch.from_numpy(train_x).type(torch.FloatTensor)
        test_x = torch.from_numpy(test_x).type(torch.FloatTensor)
        train_y = torch.from_numpy(train_y).type(torch.int64)
        test_y = torch.from_numpy(test_y).type(torch.int64)
    # reshape
    train_x = train_x.view(410, 10, 1, 120)
    test_x = test_x.view(174, 10, 1, 120)
    # train_x = train_x.view(408, 1, 10, 120)
    # test_x = test_x.view(202, 1, 10, 120)

    return train_x, test_x, train_y, test_y

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")#cuda:0

train_x, test_x, train_y, test_y = Data_Reading(Normalization=1)
train_y = train_y.squeeze()
test_y = test_y.squeeze()
train_x = train_x.to(device)
test_x = test_x.to(device)
train_y = train_y.to(device)
test_y = test_y.to(device)

cnn1 = Net()
cnn1.to(device)
cnn1.load_state_dict(torch.load('F:\\gitworkspace\\python\\e-nose-cnn\\net\\net1_86.pkl'))#F:\gitworkspace\python\e-nose-cnn\net\net1_86.pkl
te_x = Variable(test_x)
te_y = Variable(test_y)
out1 = cnn1(te_x)
predicted_test = torch.max(out1.data, 1)[1]#.data.squeeze()
total = te_y.size(0)
sum = 0
for j in range(te_y.size(0)):
    if predicted_test[j] == te_y[j]:
        sum += 1
print('predicted_test:{}, total:{}, accuracy:{}, sum:{}'.format(predicted_test, total, sum / total, sum))

predicted_test:tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 0, 2, 2, 0, 2,
        0, 0, 2, 2, 0, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3], device='cuda:0'), total:174, accuracy:0.8620689655172413, sum:150
